In [3]:
import PySimpleGUI as sg

In [4]:
sg.theme('Material 2')

'Material2'

In [5]:
text = 'Helvetica '
data = 'Courier '

title = '16'
instructions = '10'
prompt = '13'

# width of fields
dialogW = 80
textW = 15


layout = [[sg.Text('Google Shared Drive', font=text+title)],
          [sg.Text('Locate the google Shared Drive that contains the cummulative folders.', font=text+instructions)],
          [sg.Text('Current Selection:', font=text+prompt, size=(textW, 2)) , sg.Text(font=data+prompt, size=(dialogW,2), key='-SHARED DRIVE-')],
          [sg.Input(key='-SHARED DRIVE-', size=(dialogW, 1)), sg.FolderBrowse(target='-SHARED DRIVE-')],
          [sg.Button('Enter')],
          [sg.Button('Exit')]]

window = sg.Window('Google Shared Drive', layout)

while True:  # Event Loop
    event, values = window.read()
    print(event, values)
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
    if event == 'Enter':
        # Update the "output" text element to be the value of "input" element
        window['-OUTPUT-'].update(values['-SHARED DRIVE-'])

window.close()

*** Duplicate key found in your layout -SHARED DRIVE- ***
*** Replaced new key with -SHARED DRIVE-0 ***
None {'-SHARED DRIVE-0': None, 'Browse': None}


In [ ]:
import PySimpleGUI as sg

choices = ('Red', 'Green', 'Blue', 'Yellow', 'Orange', 'Purple', 'Chartreuse')

layout = [  [sg.Text('What is your favorite color?')],
            [sg.Listbox(choices, size=(15, len(choices)), key='-COLOR-', enable_events=True)] ]

window = sg.Window('Pick a color', layout)

while True:                  # the event loop
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break
    if values['-COLOR-']:    # if something is highlighted in the list
        sg.popup(f"Your favorite color is {values['-COLOR-'][0]}")
window.close()

In [10]:
def create_main_window(settings):
    sg.theme(settings['theme'])

    layout = [[sg.T('This is my main application')],
              [sg.T('Add your primary window stuff in here')],
              [sg.B('Ok'), sg.B('Exit'), sg.B('Change Settings')]]

    return sg.Window('Main Application', layout)

In [11]:
create_main_window({})

KeyError: 'theme'